这个项目的主要目的是实现一个应用，它能够在获得用户输入的，当前面临的挑战的情况下输出对应的，有助于应对这个挑战的github工具。

我们都知道现在流行的chatgpt在各个领域中表现出强大的知识储备。特别是在结合了bing搜索之后，更是能根据需求直接提供对应的网站。但是chatgpt存在一些局限性：
1. ChatGPT知识在2021年9月之前的数据进行训练，因此没有办法提供这之后的信息。
2. ChatGPT没有办法分析复杂的逻辑关系。
3. ChatGPT不能列出引用来源，其可靠性基于来源信息的可靠性，这些来源可能本身是错误的、前后矛盾的，或者经过ChatGPT组合后出现错误或矛盾。

以上三点，导致直接使用chatgpt没有办法完成本项目的目的。我们可以先看一下，在这个展示中，我们面对的是下列这个挑战：
“Geht es darum, die Sicherheit von Radwegen zu ermitteln, ist das Forschungspotential riesig. Ziel des SDSC-BW-Projekts war es, dieses zu erkunden. Ob der zahlreichen Problematiken gestaltete sich bereits die Erstellung eines Rahmenwerks schwierig. An erster Stelle standen die vielfältigen, teils verwirrenden Datenquellen – darunter unterschiedliche Websites, die Daten auf ihre eigene Weise abspeichern.”

使用chatgpt的结果：
首先是使用bing得到的回复：

<img src="./images/bing_result.png" alt="feedback from bing search" width="800" height="600">

我们可以看到，chaptgpt没办法给出有效的反馈，而它提供的网页和我们的目的也没有直接的关系。

使用ChatGPT网页对话得到的反馈会好一些：

<div style="display: inline-block;"><img src="./images/chatgpt_result.png" alt="feedback from chatgpt chatbot" width="700" height="500"></div>


虽然chatgpt4能总结出面临的挑战，但是所提供的工具针对性比较弱而且都是比较成熟的发展时间比较久了的。除此之外，会有很多额外的描述，使得回复看起来复杂。

针对chatgpt出现的分析复杂输入能力不足，回复繁复，无法提供实时的工具以及提供的链接可能出现错误等个问题，我们进行应对的核心思想是：
1. 分解需求，每次只对gpt进行简单的询问
2. 对输出进行限制，使输出简介并和主题具有联系
3. 使用github api获取最新github，保证工具的流行性和有效性

下面开始展示我们的做法，首先是加载需要的包裹

In [1]:
from git_request import search_top_starred_repositories
from gpt_request import get_response_from_chatgpt_with_context

import os
import pandas as pd
import requests

其中git_request和gpt_request都是自定义的包裹，分别用于想github服务器申请github列表和请openai申请chatgpt服务。
pandas是数据分析工具，request是网络访问工具

In [2]:
use_case = input("Please input the use case: ")

# get the theme from the user case
prompt = f"What is the theme studied in the following use case, please answer only with a keyword less then 20 letter: {use_case}"
context = []
response, context = get_response_from_chatgpt_with_context(prompt, context)
print(f"\nThe theme of the use case is: {response}\n")


Please input the use case:  Geht es darum, die Sicherheit von Radwegen zu ermitteln, ist das Forschungspotential riesig. Ziel des SDSC-BW-Projekts war es, dieses zu erkunden. Ob der zahlreichen Problematiken gestaltete sich bereits die Erstellung eines Rahmenwerks schwierig. An erster Stelle standen die vielfältigen, teils verwirrenden Datenquellen – darunter unterschiedliche Websites, die Daten auf ihre eigene Weise abspeichern.



The theme of the use case is: Radwegsicherheit



首先我们让gpt分析出对应输入的主题，我们看到gpt能够很好的完成这个工作。这里为了限制回答的简洁性，我们通过加入需求限制了gpt的输出

In [3]:
# get the challenge from the user case
prompt = f"According to the given description, what is the main problem faced by this study, please answer with 3 keywords and without explanation"
response, context = get_response_from_chatgpt_with_context(prompt, context)
print(f"The main challenge facing are: \n{response}\n")

The main challenge facing are: 
Multiple data sources, confusing data, complex framework.



然后我们要求在给定主题的情况下，总结出面临的三个挑战。因为这个挑战要应用于后续的github搜索，我们要求gpt使用关键字回复，并且不用给出解释。

In [4]:
# ask for keywords for python tools
prompt = f"I want to search for python tools for the above problem by keywords, what keywords should I use, please give me 3 suggestions and speperate them with semicolon, without explanation"
response, context = get_response_from_chatgpt_with_context(prompt, context)
keywords = response.split(";")

# list the advised git repos
for keyword in keywords:
    prompt = f"Please explain why data {keyword} is needed in the context of the use case above, and please answer in less than 50 words"
    response, context = get_response_from_chatgpt_with_context(prompt, context)
    print(response)

    git_urls, readme_urls = search_top_starred_repositories(keyword+' python')
    if git_urls is not None:
        print("For this, we recommend the following tools:")
        for git_url, readme_url in zip(git_urls, readme_urls):
            print("Repository URL:", git_url)
            print("README URL:", readme_url)
        print('-'*50)

Data scraping is needed to extract and collect data from various online sources, including websites, in order to obtain a comprehensive dataset for analyzing and assessing the safety of bike lanes.
For this, we recommend the following tools:
Repository URL: https://github.com/khuyentran1401/Data-science
README URL: https://github.com/khuyentran1401/Data-science/blob/master/README.md
Repository URL: https://github.com/stanfordjournalism/search-script-scrape
README URL: https://github.com/stanfordjournalism/search-script-scrape/blob/master/README.md
Repository URL: https://github.com/hhursev/recipe-scrapers
README URL: https://github.com/hhursev/recipe-scrapers/blob/master/README.md
Repository URL: https://github.com/scrapy/parsel
README URL: https://github.com/scrapy/parsel/blob/master/README.md
Repository URL: https://github.com/damklis/DataEngineeringProject
README URL: https://github.com/damklis/DataEngineeringProject/blob/master/README.md
--------------------------------------------

根据上面提供的每个关键字，我们从github服务器中寻找相匹配的工具，并根据其被应用的广泛性进行排序返回排名前5的结果。因为返回的结果都是通过github服务器提供的这保证了结果的可靠性。另外，因为我们是通过总结出的关键字进行匹配的，这使得得到的结果和我们的研究主题联系紧密。

github关键字搜索的依据是一下三点：
1. repository的名字和描述
2. source code和file contents
3. issue

这保证了，尽管在repository主人关键字设置出错的情况下，也能搜索到对应的repository。同时，因为关键字是由chatgpt总结出来的，这保证了得到的关键字不会出现生僻的单词。

当然，上述操作也存在缺点。比如，要求关键字必须在repository内容中出现；必须连接网络。

第一个问题我们可以通过word embedding解决。通过对关键字和repository的readme文件进行word embedding，我们可以通过比较这两个embedding的相似性来找到匹配的仓库。第二个问题可以通过本地部署gpt来解决。openai的chatgpt-4虽然还没有开源，但是已经有很多开源的替代产品。我们可以通过本地部署这些替代产品来实现我们的目的。

为此我们先加载两个自定义的方法

In [5]:
from generate_local_github_database import download_and_save_git_stared_reposiories
from git_request import search_top_related_local_repositories

download_and_save_git_stared_reposiories实现了下载指定repository信息并保存到本地的操作，search_top_related_local_repositories实现了本地gpt的部署，word embedding的实现和本地数据库的搜索。

In [6]:
# ask for keywords for python tools
prompt = f"I want to search for python tools for the above problem by keywords, what keywords should I use, please give me 3 suggestions and speperate them with semicolon, without explanation"
response, context = get_response_from_chatgpt_with_context(prompt, context)
keywords = response.split(";")

# list the advised git repos
for keyword in keywords:
    prompt = f"Please explain why data {keyword} is needed in the context of the use case above, and please answer in less than 50 words"
    response, context = get_response_from_chatgpt_with_context(prompt, context)
    print(response)

    git_urls, readme_urls = search_top_related_local_repositories(keyword, database_path = './data/repositories.csv')
    if git_urls is not None:
        print("For this, we recommend the following tools:")
        for git_url, readme_url in zip(git_urls, readme_urls):
            print("Repository URL:", git_url)
            print("README URL:", readme_url)
        print('-'*50)


Web scraping is necessary to collect data from various online sources, including websites, for analyzing the safety of bike lanes. It automates the data collection process and creates a comprehensive dataset for further analysis.


/pfs/data5/home/kit/tm/hj7422/conda/envs/vicuna/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


Using embedded DuckDB without persistence: data will be transient
Unable to connect optimized C data functions [No module named 'clickhouse_connect.driverc.buffer'], falling back to pure Python
Unable to connect ClickHouse Connect C to Numpy API [No module named 'clickhouse_connect.driverc.npconv'], falling back to pure Python


For this, we recommend the following tools:
Repository URL: https://github.com/RaRe-Technologies/gensim
README URL: https://github.com/RaRe-Technologies/gensim/blob/master/README.md
Repository URL: https://github.com/explosion/sense2vec
README URL: https://github.com/explosion/sense2vec/blob/master/README.md
Repository URL: https://github.com/BrikerMan/Kashgari
README URL: https://github.com/BrikerMan/Kashgari/blob/master/README.md
Repository URL: https://github.com/gnes-ai/gnes
README URL: https://github.com/gnes-ai/gnes/blob/master/README.md
Repository URL: https://github.com/github/semantic
README URL: https://github.com/github/semantic/blob/master/README.md
--------------------------------------------------
Data wrangling is necessary to clean, transform, and harmonize the extracted dataset from different online sources. It enables data cleaning, data manipulation, and integration to ensure consistency, quality, and accuracy of the data used for analyzing the safety of bike lanes.


Using embedded DuckDB without persistence: data will be transient


max_seq_length  512
For this, we recommend the following tools:
Repository URL: https://github.com/cpitclaudel/dBoost
README URL: https://github.com/cpitclaudel/dBoost/blob/master/README.md
Repository URL: https://github.com/amundsen-io/amundsen
README URL: https://github.com/amundsen-io/amundsen/blob/master/README.md
Repository URL: https://github.com/doccano/doccano
README URL: https://github.com/doccano/doccano/blob/master/README.md
Repository URL: https://github.com/dmlc/gluon-nlp
README URL: https://github.com/dmlc/gluon-nlp/blob/master/README.md
Repository URL: https://github.com/flairNLP/flair
README URL: https://github.com/flairNLP/flair/blob/master/README.md
--------------------------------------------------
Data visualization is necessary to present the analyzed data in a clear and visually appealing format. It helps to identify trends, patterns, and insight to stakeholders, policymakers, and residents, promoting data-driven decision-making to improve the safety of bike lanes

Using embedded DuckDB without persistence: data will be transient


max_seq_length  512
For this, we recommend the following tools:
Repository URL: https://github.com/PAIR-code/facets
README URL: https://github.com/PAIR-code/facets/blob/master/README.md
Repository URL: https://github.com/amundsen-io/amundsen
README URL: https://github.com/amundsen-io/amundsen/blob/master/README.md
Repository URL: https://github.com/bokeh/bokeh
README URL: https://github.com/bokeh/bokeh/blob/master/README.md
Repository URL: https://github.com/raghakot/keras-vis
README URL: https://github.com/raghakot/keras-vis/blob/master/README.md
Repository URL: https://github.com/yosinski/deep-visualization-toolbox
README URL: https://github.com/yosinski/deep-visualization-toolbox/blob/master/README.md
--------------------------------------------------


我们可以看到，在本地也能很好的完成工作。另外，因为我们适用的测试数据库只包含了两百个不同的repository而且种类比较集中，所以从结果上看，它的推荐结果没有联网的结果好。但是这点可以通过增加本地数据集解决。